# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
train_df = pd.read_csv(url, index_col=0)
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [2]:
train_df.shape

(6000, 61)

In [3]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular']

In [4]:
y.mean()

0.5

In [5]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
logis = LogisticRegression(solver='liblinear',C=1e9)

In [8]:
logis.fit(X_train,y_train)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [9]:
from sklearn import metrics

In [10]:
y_pred = logis.predict(X_test)

In [11]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.61      0.67      0.64       744
           1       0.64      0.57      0.60       756

   micro avg       0.62      0.62      0.62      1500
   macro avg       0.62      0.62      0.62      1500
weighted avg       0.62      0.62      0.62      1500



In [12]:
from sklearn.tree import DecisionTreeClassifier

In [13]:
DTC = DecisionTreeClassifier(random_state=1)

In [14]:
DTC.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [15]:
y_pred1 = DTC.predict(X_test)

In [16]:
print(metrics.classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.54      0.55      0.55       744
           1       0.55      0.53      0.54       756

   micro avg       0.54      0.54      0.54      1500
   macro avg       0.54      0.54      0.54      1500
weighted avg       0.54      0.54      0.54      1500



# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
import numpy as np

In [18]:
models = {'log': LogisticRegression(solver='liblinear',C=1e9),
          'dt1': DecisionTreeClassifier(max_depth=None),
          'dt2': DecisionTreeClassifier(max_depth=2)}

In [19]:
y_pred = pd.DataFrame (columns =models.keys(),index= y_test.index )

In [20]:
#for tp in models.keys():
for i, model  in models.items():
       
    bagreg = BaggingClassifier(model, n_estimators=100,bootstrap=True, oob_score=True, random_state=1,n_jobs=-1)
    
    bagreg.fit(X_train, y_train)
    
    y_pred[i] = bagreg.predict(X_test)
    

        

In [21]:
y_pred.head(10)

,log,dt1,dt2
1483,1,1,1
2185,1,1,1
2520,1,1,1
3721,1,1,1
3727,0,0,0
4524,0,1,1
234,0,0,0
4735,0,0,0
5839,0,0,0
2939,1,0,1


# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [22]:
metrics.f1_score(np.round(y_pred.mean(axis=1)).astype(int), y_test), metrics.accuracy_score(np.round(y_pred.mean(axis=1)).astype(int), y_test)

(0.6387575962187712, 0.6433333333333333)

In [23]:
from sklearn.ensemble import VotingClassifier

In [24]:
alg1 = LogisticRegression(solver='liblinear',C=1e9)
alg2 = DecisionTreeClassifier(max_depth=None)
alg3 = DecisionTreeClassifier(max_depth=2)

bagreg1 = BaggingClassifier(alg1, n_estimators=100,bootstrap=True, oob_score=True,n_jobs=-1, random_state= 1)
bagreg2 = BaggingClassifier(alg2, n_estimators=100,bootstrap=True, oob_score=True,n_jobs=-1, random_state= 1)
bagreg3 = BaggingClassifier(alg3, n_estimators=100,bootstrap=True, oob_score=True,n_jobs=-1, random_state= 1)

Alg = VotingClassifier(estimators=[('LR',bagreg1),('DT',bagreg2),('DT2',bagreg3)], voting='hard')
Alg.fit(X_train,y_train)

VotingClassifier(estimators=[('LR', BaggingClassifier(base_estimator=LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001,...n_estimators=100, n_jobs=-1, oob_score=True,
         random_state=1, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

In [25]:
y_pred0 = Alg.predict(X_test)

In [26]:
metrics.f1_score(y_pred0, y_test), metrics.accuracy_score(y_pred0, y_test)

(0.6387575962187712, 0.6433333333333333)

# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

In [27]:
test = pd.DataFrame(y_pred0, columns=['g'])

In [28]:
popular = pd.crosstab(test.g, columns = 'count')

In [29]:
popular/popular.sum()

col_0,count
g,
0,0.516667
1,0.483333


In [30]:
alg1 = LogisticRegression(solver='liblinear',C=1e9)
alg2 = DecisionTreeClassifier(max_depth=None)
alg3 = DecisionTreeClassifier(max_depth=2)

bagreg1 = BaggingClassifier(alg1, n_estimators=100,bootstrap=True, oob_score=True,n_jobs=-1, random_state= 1)
bagreg2 = BaggingClassifier(alg2, n_estimators=100,bootstrap=True, oob_score=True,n_jobs=-1, random_state= 1)
bagreg3 = BaggingClassifier(alg3, n_estimators=100,bootstrap=True, oob_score=True,n_jobs=-1, random_state= 1)

Alg0 = VotingClassifier(estimators=[('LR',bagreg1),('DT',bagreg2),('DT2',bagreg3)], voting='soft')
Alg0.fit(X_train,y_train)

VotingClassifier(estimators=[('LR', BaggingClassifier(base_estimator=LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001,...n_estimators=100, n_jobs=-1, oob_score=True,
         random_state=1, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='soft', weights=None)

In [31]:
prob = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
thre_prob = pd.DataFrame(index=y_test.index, columns=prob)
Results = pd.DataFrame(columns=prob,index=['h'])

for i in prob:
    thre_prob[i] = (Alg0.predict_proba(X_test)[:,1]>= i).astype(np.int)
    Results[i] = metrics.f1_score(thre_prob[i], y_test)
Results

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8
h,0.670213,0.670213,0.693496,0.702584,0.642857,0.51453,0.259136,0.028534


In [32]:
metrics.f1_score((Alg0.predict_proba(X_test)[:,1]>= 0.3), y_test), metrics.accuracy_score((Alg0.predict_proba(X_test)[:,1]>= 0.3), y_test)

(0.6934955545156761, 0.5633333333333334)

# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [33]:
bagreg1.fit(X_train, y_train)
y_pred = bagreg1.predict(X_test)
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.6085136078157711, 0.626)

In [34]:
errors = np.zeros(bagreg1.n_estimators)
y_pred_all_ = np.zeros((X_test.shape[0], bagreg1.n_estimators))

for i in range(bagreg1.n_estimators):
    oob_sample = ~bagreg1.estimators_samples_[i]
    y_pred_ = bagreg1.estimators_[i].predict(X_train.values[oob_sample])
    errors[i] = metrics.accuracy_score(y_pred_, y_train.values[oob_sample])
    y_pred_all_[:, i] = bagreg1.estimators_[i].predict(X_test)
    
alpha = (1 - errors) / (1 - errors).sum()
bag1 = pd.DataFrame((np.sum(y_pred_all_ * alpha, axis=1) >= 0.5).astype(np.int),columns=['bag1'])
bag11 = pd.DataFrame(np.sum(y_pred_all_ * alpha, axis=1),columns=['bag1'])

In [35]:
bagreg2.fit(X_train, y_train)
y_pred = bagreg2.predict(X_test)
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.635752688172043, 0.6386666666666667)

In [36]:
errors = np.zeros(bagreg2.n_estimators)
y_pred_all_ = np.zeros((X_test.shape[0], bagreg2.n_estimators))

for i in range(bagreg2.n_estimators):
    oob_sample = ~bagreg2.estimators_samples_[i]
    y_pred_ = bagreg2.estimators_[i].predict(X_train.values[oob_sample])
    errors[i] = metrics.accuracy_score(y_pred_, y_train.values[oob_sample])
    y_pred_all_[:, i] = bagreg2.estimators_[i].predict(X_test)
    
alpha = (1 - errors) / (1 - errors).sum()
bag2 = pd.DataFrame((np.sum(y_pred_all_ * alpha, axis=1) >= 0.5).astype(np.int),columns=['bag2'])
bag22 = pd.DataFrame(np.sum(y_pred_all_ * alpha, axis=1),columns=['bag2'])

In [37]:
bagreg3.fit(X_train, y_train)
y_pred = bagreg3.predict(X_test)
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.6434782608695653, 0.6446666666666667)

In [38]:
errors = np.zeros(bagreg3.n_estimators)
y_pred_all_ = np.zeros((X_test.shape[0], bagreg3.n_estimators))

for i in range(bagreg3.n_estimators):
    oob_sample = ~bagreg3.estimators_samples_[i]
    y_pred_ = bagreg3.estimators_[i].predict(X_train.values[oob_sample])
    errors[i] = metrics.accuracy_score(y_pred_, y_train.values[oob_sample])
    y_pred_all_[:, i] = bagreg3.estimators_[i].predict(X_test)
    
alpha = (1 - errors) / (1 - errors).sum()
bag3 = pd.DataFrame((np.sum(y_pred_all_ * alpha, axis=1) >= 0.5).astype(np.int),columns=['bag3'])
bag33 = pd.DataFrame(np.sum(y_pred_all_ * alpha, axis=1),columns=['bag3'])

In [39]:
Yresul = pd.concat([bag1, bag2,bag3], axis=1, sort=False)

In [40]:
Yresul.head(10)

,bag1,bag2,bag3
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,0,0,0
5,0,1,1
6,0,0,0
7,0,0,0
8,0,0,0
9,1,0,1


In [41]:
metrics.f1_score(np.round(Yresul.mean(axis=1)).astype(int), y_test), metrics.accuracy_score(np.round(Yresul.mean(axis=1)).astype(int), y_test)

(0.6411290322580645, 0.644)

# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [42]:
Yest = np.round(Yresul.mean(axis=1)).astype(int)

In [43]:
popular = pd.crosstab(Yest, columns = 'count')

In [44]:
popular/popular.sum()

col_0,count
row_0,
0,0.512
1,0.488


In [45]:
Yresul2 = pd.concat([bag11, bag22,bag33], axis=1, sort=False)

In [46]:
Yresul2.head(10)

,bag1,bag2,bag3
0,1.000000,0.830031,1.000000
1,1.000000,0.651194,1.000000
2,1.000000,0.538299,1.000000
3,1.000000,0.652927,1.000000
4,0.000000,0.278354,0.010060
5,0.000000,0.543207,0.969114
6,0.000000,0.140325,0.030293
7,0.000000,0.409385,0.000000
8,0.480895,0.490891,0.039988
9,1.000000,0.471120,0.990096


In [47]:
Yprue = Yresul2.mean(axis=1)
metrics.f1_score((Yprue >= 0.5).astype(np.int) , y_test)

0.6256906077348066

In [48]:
prob = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
thre_prob = pd.DataFrame(index=Yprue.index, columns=prob)
Results = pd.DataFrame(columns=prob,index=['h'])

for i in prob:
    thre_prob[i] = (Yprue >= i).astype(np.int)
    
    Results[i] = metrics.f1_score(thre_prob[i], y_test)
    
Results

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8
h,0.696532,0.680304,0.669127,0.656831,0.625691,0.58963,0.57406,0.527183


In [49]:
metrics.f1_score(((Yprue >= 0.1).astype(np.int)), y_test), metrics.accuracy_score(((Yprue >= 0.1).astype(np.int)), y_test)

(0.6965317919075144, 0.58)

# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score

In [50]:
import numpy as np

# set a seed for reproducibility
np.random.seed(123)

n_samples = X_train.shape[0]
n_B = 100

# create ten bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]
#samples
X_train.iloc[samples[0], :].head()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
2239,142.0,11.0,135.0,0.631579,1.0,0.690000,11.0,4.0,8.0,0.0,...,0.354762,0.285714,0.5,-0.308333,-0.625,-0.100000,0.333333,0.250000,0.166667,0.250000
4838,349.0,10.0,1411.0,0.431259,1.0,0.630996,7.0,6.0,4.0,1.0,...,0.438312,0.050000,1.0,-0.322500,-0.800,-0.100000,1.000000,0.100000,0.500000,0.100000
1047,731.0,12.0,238.0,0.621277,1.0,0.823077,3.0,1.0,1.0,0.0,...,0.291259,0.100000,0.6,0.000000,0.000,0.000000,0.000000,0.000000,0.500000,0.000000
373,583.0,8.0,253.0,0.661355,1.0,0.846667,7.0,0.0,0.0,1.0,...,0.432149,0.050000,1.0,-0.247194,-0.500,-0.071429,0.400000,-0.400000,0.100000,0.400000
2595,659.0,15.0,458.0,0.512088,1.0,0.662879,5.0,3.0,1.0,0.0,...,0.368586,0.136364,0.8,-0.219167,-0.500,-0.125000,0.454545,0.136364,0.045455,0.136364


In [51]:
models

{'log': LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
 'dt1': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'),
 'dt2': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             spl

In [52]:
y_pred_df = pd.DataFrame(index=y_test.index, columns=[list(range(300))])
X_train_2 = pd.DataFrame(index=y_train.index, columns=[list(range(300))])

for i, sample in enumerate(samples):
    X_train = X_train.iloc[sample]
    y_train = y_train.iloc[sample]
       
    #Se calcula el resultado de los arboles con overfit
    X_train_2[i] = bagreg1.predict(X_train)
    X_train_2[i+100] = bagreg2.predict(X_train)
    X_train_2[i+200] = bagreg3.predict(X_train)
    
    #resultado del predict con test
    y_pred_df[i] = bagreg1.predict(X_test)
    y_pred_df[i+100] = bagreg2.predict(X_test)
    y_pred_df[i+200] = bagreg3.predict(X_test)

In [54]:
from sklearn.linear_model import LogisticRegressionCV
lr = LogisticRegressionCV(cv = 50 )
lr.fit(X_train_2, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=50, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

In [55]:
y_predict_lr = pd.DataFrame(lr.predict(y_pred_df), columns=['g'])

In [56]:
metrics.f1_score(y_predict_lr, y_test), metrics.accuracy_score(y_predict_lr, y_test)

(0.635752688172043, 0.6386666666666667)

In [60]:
prob = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
thre_prob = pd.DataFrame(index=y_test.index, columns=prob)
Results = pd.DataFrame(columns=prob,index=['h'])

for i in prob:
    thre_prob[i] = (lr.predict_proba(y_pred_df)[:,1]>= i).astype(np.int)
    Results[i] = metrics.f1_score(thre_prob[i], y_test)
Results

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8
h,0.670213,0.663385,0.635753,0.635753,0.635753,0.635753,0.635753,0.570988
